In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import statistics
from scipy.stats import norm, t
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab
import scipy

In [2]:
#Annual VaR and CVar Engine Utilizng the Variance and Covariance Method
list_assets = ["HSBC", "BCS", "LYG", "RBSPF"]
dict_assets = {}

def data_grabber(tickers, assets):
    #Function designed to take any ticker and grab annual data
    for x in tickers:
        try: 
            
            ticker = yf.Ticker(x)
            ticker_data = ticker.history(period = "1y", interval = "1d")
            
            if not ticker_data.empty:
                assets[x] = ticker_data
            else:
                print(f"The associated data for {x} is empty")
        except Exception as e:
            print(f"There was an error in fetching {x} data")
            
    return assets

def returns(dict_assets):
    #Calculates daily returns of each asset in the portfolio and puts them into a seperate dataframe
    for x in dict_assets:
        df_common = dict_assets[x]    
        df_common["Returns"] = df_common["Close"].pct_change()
        df_common = df_common.dropna()        
        dict_assets[x] = df_common
    
    dict_returns = {}    
    for x in dict_assets:
        df_common = dict_assets[x]
        series_returns = df_common["Returns"]
        dict_returns[x] = series_returns
        
    df_returns = pd.concat(dict_returns.values(), axis = 1, keys=dict_returns)
    mean_returns = df_returns.mean()
    
    return dict_assets, df_returns, mean_returns

def weights(dict_assets):
    #Random weight generator 
    array_weights = np.random.random(len(dict_assets.keys()))
    array_weights /= np.sum(array_weights)

    counter = 0

    for key in dict_assets:
        df_common = dict_assets[key]
        df_common["Weights"] = array_weights[counter]
        df_common = df_common.dropna()
        dict_assets[key] = df_common
        counter = counter + 1
        
    return dict_assets, array_weights

def portfolio(dict_assets):
    #Assembles the portfolio into a singular dataframe
    df_portfolio = pd.concat(dict_assets.values(), axis = 1, keys=dict_assets.keys())
    df_portfolio = df_portfolio.reset_index()
    df_portfolio['Date'] = df_portfolio['Date'].dt.tz_convert('America/New_York').dt.tz_localize(None)

    return df_portfolio
    
def portfolio_return(df_portfolio, df_returns, array_weights):
    #Calculates the daily returns of the portfolio given the weights
        
    series_portfolio_return = df_returns.dot(array_weights)
    df_portfolio["Portfolio Returns"] = series_portfolio_return
    
    return df_portfolio, df_returns, series_portfolio_return
    
def covariance_matrix(dict_assets, df_returns):
    #Finds the covariance between the returns of each asset in the portfolio        
    df_cov = df_returns.cov()
    
    return df_cov
  
def portfolio_variance_calculator(df_cov, array_weights):
    #Find portfolio variance given covariance matrix and weights of the portfolio
    array_portfolio_variance = np.dot(array_weights.T, np.dot(df_cov, array_weights))
    float_portfolio_variance = float(array_portfolio_variance)
    return float_portfolio_variance  
    
def portfolio_std(float_portfolio_variance):
    #Multiply by the square root of 252 to annualize the volatility
    float_portfolio_std = (math.sqrt(float_portfolio_variance))*np.sqrt(252)
    
    return float_portfolio_std
    
def portfolio_mean_return(mean_returns, array_weights):
    #Multiply by the number of trading days to annualize average portfolio return
    P_returns = np.sum(mean_returns*array_weights)*252

    
    return P_returns

def Var_Parametric(portfolio_return, std, distribution, dof = 6):
    """Calculate the portfolio VaR Given a distribution with known parameters"""
    list_alpha = [0.9, 0.95, 0.975, 0.99]
    dict_VaR = {}
    
    for x in list_alpha:
        if distribution == "normal":
            VaR = norm.ppf(1-x) * std - portfolio_return
        elif distribution == "t-distribution":
            nu = dof
            VaR = np.sqrt((nu-2)/nu) * t.ppf(1-x, nu)* std - portfolio_return
        else:
            raise TypeError("Expected distribution to be normal or t-distribution")
        dict_VaR[x] = VaR
    print(f"Annual Var with associated confidence levels: {dict_VaR}")    
        
    return dict_VaR

def CVar_Parametric(portfolio_return, std, distribution = "normal", alpha = 5, dof = 6):
    """Calculate the portfolio CVaR Given a distribution with known parameters"""
    list_alpha = [0.9, 0.95, 0.975, 0.99]
    dict_CVaR = {}
    
    for x in list_alpha:
        if distribution == "normal":
            CVaR = (1-x)**-1 * norm.pdf(norm.ppf(1-x))* std - portfolio_return
        elif distribution == "t-distribution":
            nu = dof
            x_anu = t.ppf(1-x, nu)
            CVaR = -1/(1-x) * (1-nu)**-1 * (nu-2+x_anu**2) * t.ppf(x_anu, nu)* std - portfolio_return
        else:
            raise TypeError("Expected distribution to be normal or t-distribution")
        dict_CVaR[x] = CVaR
    print(f"Annual CVar with associated confidence levels: {dict_CVaR}")    
        
    return dict_CVaR

In [3]:
data_grabber(list_assets, dict_assets)
dict_assets, df_returns, mean_returns = returns(dict_assets)
df_portfolio = portfolio(dict_assets)
dict_assets, array_weights = weights(dict_assets)
df_portfolio, df_returns, series_portfolio_returns = portfolio_return(df_portfolio, df_returns, array_weights)
df_cov = covariance_matrix(dict_assets, df_returns)
portfolio_variance = portfolio_variance_calculator(df_cov, array_weights)
std = portfolio_std(portfolio_variance)
Pret = portfolio_mean_return(mean_returns, array_weights)
Var_Parametric(Pret, std, "normal")

Annual Var with associated confidence levels: {0.9: -0.2728494593312391, 0.95: -0.3604823068982307, 0.975: -0.4364907331273753, 0.99: -0.5248669548732412}


{0.9: -0.2728494593312391,
 0.95: -0.3604823068982307,
 0.975: -0.4364907331273753,
 0.99: -0.5248669548732412}